In [1]:
import numpy as np 
import pandas as pd 

In [2]:
# Show all column in head
pd.set_option('display.max_columns', None)

In [3]:
customer_demographics_df = pd.read_csv('customer_demographics.csv')
campaign_df = pd.read_csv('campaign_data.csv')

item_data_df = pd.read_csv('item_data.csv')
customer_transaction_df = pd.read_csv('customer_transaction_data.csv')

coupon_item_mapping_df = pd.read_csv('coupon_item_mapping.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_QyjYwdj.csv')

In [4]:
print(train_df.shape)
print(test_df.shape)


(78369, 5)
(50226, 4)


In [5]:
customer_demographics_df.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [6]:
# Todo replace nan with mode
customer_demographics_df['marital_status'].fillna('Unknown',inplace=True)
customer_demographics_df['family_size'].fillna(-1,inplace=True)
customer_demographics_df['no_of_children'].fillna(-1,inplace=True)

for index, row in customer_demographics_df.iterrows():
    family_size = row['family_size']
    no_of_children = row['no_of_children']
    
    if (row['family_size']=='5+'):
        customer_demographics_df.at[index, 'family_size'] = 5
        family_size = 5
    if (row['no_of_children']=='3+'):
        customer_demographics_df.at[index, 'no_of_children'] = 3
        no_of_children = 3
        
    family_size = float(family_size)
    no_of_children = float(no_of_children)
    
    #print(type(row['marital_status']))
    if (row['marital_status']=='Unknown') :
        if (family_size>=2):
            customer_demographics_df.at[index, 'marital_status'] = 'Married'
        elif (family_size==1) :
            customer_demographics_df.at[index, 'marital_status'] = 'Single'
        elif (no_of_children>0) :
            customer_demographics_df.at[index, 'marital_status'] = 'Married'
        else :
            customer_demographics_df.at[index, 'marital_status'] = 'Unknown'
            
    if (row['no_of_children']==-1) :
        if (family_size>=2):
            customer_demographics_df.at[index, 'no_of_children'] = family_size-2
        else:
            customer_demographics_df.at[index, 'no_of_children'] = 0

In [7]:
customer_demographics_df.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,0,4
1,6,46-55,Married,0,2,0,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,0,5


In [8]:
train_df = pd.merge(train_df, campaign_df, how='left', on='campaign_id')
test_df = pd.merge(test_df, campaign_df, how='left', on='campaign_id')

train_df = pd.merge(train_df, customer_demographics_df, how='left', on='customer_id')
test_df = pd.merge(test_df, customer_demographics_df, how='left', on='customer_id')



In [9]:
print(train_df.shape)
print(test_df.shape)
train_df.head()

(78369, 14)
(50226, 13)


,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0,X,19/05/13,05/07/13,46-55,Single,0.0,1,0,5.0
1,2,13,116,48,0,X,19/05/13,05/07/13,36-45,Married,0.0,2,0,3.0
2,6,9,635,205,0,Y,11/03/13,12/04/13,46-55,Married,0.0,2,0,7.0
3,7,13,644,1050,0,X,19/05/13,05/07/13,NaN,NaN,NaN,NaN,NaN,NaN
4,9,8,1017,1489,0,X,16/02/13,05/04/13,46-55,Married,0.0,2,0,3.0


In [10]:
coupon_item_mapping_df.shape

(92663, 2)

In [11]:
def buildGroupName(brand, brand_type, category):
    return str(brand) + '_' + brand_type + '_' + category
    
item_data_df['group_name'] =  item_data_df.apply(lambda x: buildGroupName(x.brand, x.brand_type, x.category), axis=1)

In [12]:
groupByMultiple = item_data_df.groupby(['group_name'])
result = groupByMultiple.item_id.min()
col_encoded = result.to_dict()   
item_data_df['item_group_id'] = item_data_df['group_name'].map(col_encoded)

In [13]:
item_data_df.shape

(74066, 6)

In [14]:
item_data_df.head()

,item_id,brand,brand_type,category,group_name,item_group_id
0,1,1,Established,Grocery,1_Established_Grocery,1
1,2,1,Established,Miscellaneous,1_Established_Miscellaneous,2
2,3,56,Local,Bakery,56_Local_Bakery,3
3,4,56,Local,Grocery,56_Local_Grocery,4
4,5,56,Local,Grocery,56_Local_Grocery,4


In [15]:
coupon_item_mapping_df.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [16]:
coupon_item_map_df= pd.merge(coupon_item_mapping_df, item_data_df, how='left', on='item_id')
coupon_item_map_df.shape

(92663, 7)

In [17]:
coupon_item_map_df.isnull().sum()

coupon_id        0
item_id          0
brand            0
brand_type       0
category         0
group_name       0
item_group_id    0
dtype: int64

In [18]:
print('Duplicated rows count', customer_transaction_df.duplicated().sum())
#Dropping duplicated rows from log_df
customer_transaction_df = customer_transaction_df.drop_duplicates(keep='first')
print(customer_transaction_df.shape)

Duplicated rows count 2916
(1321650, 7)


In [19]:
customer_transaction_df.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [20]:
customer_transaction_df.isnull().sum()

date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64

In [21]:
customer_transaction_df= pd.merge(coupon_item_map_df, customer_transaction_df, how='inner', on='item_id')
customer_transaction_df.shape

(2095709, 13)

In [22]:
customer_transaction_df.isnull().sum()

coupon_id          0
item_id            0
brand              0
brand_type         0
category           0
group_name         0
item_group_id      0
date               0
customer_id        0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64

In [23]:
for df in [train_df, test_df]:
    df['age_range'].fillna(df['age_range'].mode()[0], inplace=True)
    df['marital_status'].fillna(df['marital_status'].mode()[0], inplace=True)
    df['rented'].fillna(df['rented'].mode()[0], inplace=True)
    df['family_size'].fillna(df['family_size'].mode()[0], inplace=True)
    df['no_of_children'].fillna(df['no_of_children'].mode()[0], inplace=True)
    df['income_bracket'].fillna(df['income_bracket'].mode()[0], inplace=True)

In [24]:
train_df['start_date'] = pd.to_datetime(train_df['start_date'], format='%d/%m/%y')
train_df['end_date'] = pd.to_datetime(train_df['end_date'], format='%d/%m/%y')

test_df['start_date'] = pd.to_datetime(test_df['start_date'], format='%d/%m/%y')
test_df['end_date'] = pd.to_datetime(test_df['end_date'], format='%d/%m/%y')

customer_transaction_df['date'] = pd.to_datetime(customer_transaction_df['date'], format='%Y-%m-%d')

# Create new column
def calculateDays (start, end):    
    return float((end-start).days)

train_df['campaign_duration'] = train_df.apply(lambda x: calculateDays(x.start_date, x.end_date), axis=1)
test_df['campaign_duration'] = test_df.apply(lambda x: calculateDays(x.start_date, x.end_date), axis=1)

In [25]:
for k in train_df.columns:
    print(k,train_df[k].nunique())
    
train_df.head()

id 78369
campaign_id 18
coupon_id 866
customer_id 1428
redemption_status 2
campaign_type 2
start_date 17
end_date 17
age_range 6
marital_status 2
rented 2
family_size 5
no_of_children 4
income_bracket 12
campaign_duration 11


,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_duration
0,1,13,27,1053,0,X,2013-05-19,2013-07-05,46-55,Single,0.0,1,0,5.0,47.0
1,2,13,116,48,0,X,2013-05-19,2013-07-05,36-45,Married,0.0,2,0,3.0,47.0
2,6,9,635,205,0,Y,2013-03-11,2013-04-12,46-55,Married,0.0,2,0,7.0,32.0
3,7,13,644,1050,0,X,2013-05-19,2013-07-05,46-55,Married,0.0,2,0,5.0,47.0
4,9,8,1017,1489,0,X,2013-02-16,2013-04-05,46-55,Married,0.0,2,0,3.0,48.0


In [28]:

customer_transaction_df['category'] = customer_transaction_df['category'].astype('category')
customer_transaction_df['brand_type'] = customer_transaction_df['brand'].astype('category')
customer_transaction_df['group_name'] = customer_transaction_df['group_name'].astype('category')

In [29]:

cat_agg=['count','nunique']
num_agg=['min','mean','max','sum']
agg_col={
    'group_name':cat_agg, 'customer_id':cat_agg, 'item_id':cat_agg,'selling_price':num_agg,'coupon_discount':num_agg,
       'other_discount':num_agg,'category':['count','nunique','mean'], 'brand_type':['count','nunique','mean']
}

for k in customer_transaction_df.columns:
    if k.startswith('group_name') or k.startswith('customer_id') or k.startswith('item_id'):
        #agg_col[k]=['sum','mean']
         agg_col[k]=cat_agg
    elif k.startswith('date'):
        agg_col[k]=cat_agg
    elif k.startswith('selling_price') or k.startswith('coupon_discount') or k.startswith('other_discount'):
        agg_col[k]=num_agg
    elif k.startswith('brand_type') or k.startswith('category') or k.startswith('category'):
        agg_col[k]=cat_agg
        
agg_col



{'group_name': ['count', 'nunique'],
 'customer_id': ['count', 'nunique'],
 'item_id': ['count', 'nunique'],
 'selling_price': ['min', 'mean', 'max', 'sum'],
 'coupon_discount': ['min', 'mean', 'max', 'sum'],
 'other_discount': ['min', 'mean', 'max', 'sum'],
 'category': ['count', 'nunique'],
 'brand_type': ['count', 'nunique'],
 'date': ['count', 'nunique']}

In [ ]:

customer_transaction_df=customer_transaction_df.groupby('customer_id','item_id').agg(agg_col)

In [31]:
customer_transaction_df.head()

group_name         customer_id         item_id          \
                 count nunique       count nunique   count nunique   
customer_id                                                          
1                 1665      82        1665       1    1665     211   
2                  672     108         672       1     672     216   
3                 1040      76        1040       1    1040     223   
4                  360      51         360       1     360      74   
5                 1301     117        1301       1    1301     289   

            selling_price                                coupon_discount  \
                      min        mean     max        sum             min   
customer_id                                                                
1                   21.37   98.212006  445.25  163522.99          -89.05   
2                   17.45  104.340164  640.80   70116.59          -89.05   
3                    8.90  120.519029  862.00  125339.79          -71.24   
4                   34.55  136.391194  623.35   49100.83          -89.05   
5                   13.89  125.204174  605.18  162890.63          -71.24   

                                    other_discount                            \
                 mean  max      sum            min       mean  max       sum   
customer_id                                                                    
1           -1.914619  0.0 -3187.84        -372.23 -23.723177  0.0 -39499.09   
2           -0.318036  0.0  -213.72        -312.03 -18.483289  0.0 -12420.77   
3           -2.184712  0.0 -2272.10        -463.06 -26.191000  0.0 -27238.64   
4           -0.247361  0.0   -89.05        -390.04 -18.571528  0.0  -6685.75   
5           -0.154689  0.0  -201.25        -216.21 -13.840138  0.0 -18006.02   

            category         brand_type          date          
               count nunique      count nunique count nunique  
customer_id                                                    
1               1665       6       1665      76  1665      52  
2                672       8        672     103   672      34  
3               1040       7       1040      71  1040      37  
4                360       7        360      48   360      25  
5               1301      12       1301     108  1301     143

In [32]:
customer_transaction_df.columns=['J_' + '_'.join(col).strip() for col in customer_transaction_df.columns.values]
customer_transaction_df.reset_index(inplace=True)
customer_transaction_df.head()

,customer_id,J_group_name_count,J_group_name_nunique,J_customer_id_count,J_customer_id_nunique,J_item_id_count,J_item_id_nunique,J_selling_price_min,J_selling_price_mean,J_selling_price_max,J_selling_price_sum,J_coupon_discount_min,J_coupon_discount_mean,J_coupon_discount_max,J_coupon_discount_sum,J_other_discount_min,J_other_discount_mean,J_other_discount_max,J_other_discount_sum,J_category_count,J_category_nunique,J_brand_type_count,J_brand_type_nunique,J_date_count,J_date_nunique
0,1,1665,82,1665,1,1665,211,21.37,98.212006,445.25,163522.99,-89.05,-1.914619,0.0,-3187.84,-372.23,-23.723177,0.0,-39499.09,1665,6,1665,76,1665,52
1,2,672,108,672,1,672,216,17.45,104.340164,640.80,70116.59,-89.05,-0.318036,0.0,-213.72,-312.03,-18.483289,0.0,-12420.77,672,8,672,103,672,34
2,3,1040,76,1040,1,1040,223,8.90,120.519029,862.00,125339.79,-71.24,-2.184712,0.0,-2272.10,-463.06,-26.191000,0.0,-27238.64,1040,7,1040,71,1040,37
3,4,360,51,360,1,360,74,34.55,136.391194,623.35,49100.83,-89.05,-0.247361,0.0,-89.05,-390.04,-18.571528,0.0,-6685.75,360,7,360,48,360,25
4,5,1301,117,1301,1,1301,289,13.89,125.204174,605.18,162890.63,-71.24,-0.154689,0.0,-201.25,-216.21,-13.840138,0.0,-18006.02,1301,12,1301,108,1301,143


In [33]:
customer_transaction_df.shape

(1582, 25)

In [35]:
train_df.shape

(78369, 15)

In [36]:
train_df['start_date'] = pd.to_datetime(train_df['start_date'], format='%d/%m/%y')
train_df['end_date'] = pd.to_datetime(train_df['end_date'], format='%d/%m/%y')

test_df['start_date'] = pd.to_datetime(test_df['start_date'], format='%d/%m/%y')
test_df['end_date'] = pd.to_datetime(test_df['end_date'], format='%d/%m/%y')

# Create new column
def calculateDays (start, end):    
    return float((end-start).days)

train_df['campaign_duration'] = train_df.apply(lambda x: calculateDays(x.start_date, x.end_date), axis=1)
test_df['campaign_duration'] = test_df.apply(lambda x: calculateDays(x.start_date, x.end_date), axis=1)


In [37]:
del train_df['start_date']
del train_df['end_date']

del test_df['start_date']
del test_df['end_date']

In [38]:
train_df['rented'] = train_df['rented'].astype('category')
test_df['rented'] = test_df['rented'].astype('category')

train_df['family_size'] = train_df['family_size'].astype('float64')
test_df['family_size'] = test_df['family_size'].astype('float64')

train_df['no_of_children'] = train_df['no_of_children'].astype('float64')
test_df['no_of_children'] = test_df['no_of_children'].astype('float64')

train_df['income_bracket'] = train_df['income_bracket'].astype('category')
test_df['income_bracket'] = test_df['income_bracket'].astype('category')

In [39]:
train_temp_df= train_df
test_temp_df= test_df

In [40]:
train_df = pd.merge(train_df, customer_transaction_df, how='left', on='customer_id')
test_df = pd.merge(test_df, customer_transaction_df, how='left', on='customer_id')

In [41]:
print(train_df.shape)
print(test_df.shape)

(78369, 37)
(50226, 36)


In [42]:
train_df.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_duration,J_group_name_count,J_group_name_nunique,J_customer_id_count,J_customer_id_nunique,J_item_id_count,J_item_id_nunique,J_selling_price_min,J_selling_price_mean,J_selling_price_max,J_selling_price_sum,J_coupon_discount_min,J_coupon_discount_mean,J_coupon_discount_max,J_coupon_discount_sum,J_other_discount_min,J_other_discount_mean,J_other_discount_max,J_other_discount_sum,J_category_count,J_category_nunique,J_brand_type_count,J_brand_type_nunique,J_date_count,J_date_nunique
0,1,13,27,1053,0,X,46-55,Single,0.0,1.0,0.0,5.0,47.0,439,49,439,1,439,109,24.58,148.957836,2714.24,65392.49,-89.05,-0.202847,0.0,-89.05,-745.88,-43.426788,0.0,-19064.36,439,5,439,45,439,55
1,2,13,116,48,0,X,36-45,Married,0.0,2.0,0.0,3.0,47.0,671,68,671,1,671,146,10.33,109.484113,445.25,73463.84,-89.05,-0.636990,0.0,-427.42,-372.23,-32.396811,0.0,-21738.26,671,7,671,64,671,54
2,6,9,635,205,0,Y,46-55,Married,0.0,2.0,0.0,7.0,32.0,1695,133,1695,1,1695,325,17.45,122.973457,1056.13,208440.01,-89.05,-2.161740,0.0,-3664.15,-468.05,-20.369322,0.0,-34526.00,1695,11,1695,122,1695,75
3,7,13,644,1050,0,X,46-55,Married,0.0,2.0,0.0,5.0,47.0,345,44,345,1,345,113,13.89,101.946986,708.48,35171.71,-89.05,-0.516232,0.0,-178.10,-312.03,-20.804812,0.0,-7177.66,345,5,345,42,345,49
4,9,8,1017,1489,0,X,46-55,Married,0.0,2.0,0.0,3.0,48.0,721,64,721,1,721,175,14.25,99.177476,449.52,71506.96,-47.73,-0.528128,0.0,-380.78,-284.96,-24.672288,0.0,-17788.72,721,5,721,59,721,74


In [43]:
test_df.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_duration,J_group_name_count,J_group_name_nunique,J_customer_id_count,J_customer_id_nunique,J_item_id_count,J_item_id_nunique,J_selling_price_min,J_selling_price_mean,J_selling_price_max,J_selling_price_sum,J_coupon_discount_min,J_coupon_discount_mean,J_coupon_discount_max,J_coupon_discount_sum,J_other_discount_min,J_other_discount_mean,J_other_discount_max,J_other_discount_sum,J_category_count,J_category_nunique,J_brand_type_count,J_brand_type_nunique,J_date_count,J_date_nunique
0,3,22,869,967,X,36-45,Single,0.0,1.0,0.0,5.0,32.0,2391,137,2391,1,2391,380,7.12,117.916859,782.93,281939.21,-89.05,-2.146708,0.0,-5132.78,-142.48,-14.503501,0.0,-34677.87,2391,9,2391,124,2391,64
1,4,20,389,1566,Y,26-35,Married,0.0,2.0,0.0,9.0,70.0,4678,193,4678,1,4678,761,3.21,111.108420,1655.62,519765.19,-102.94,-0.536881,0.0,-2511.53,-156.02,-11.821556,0.0,-55301.24,4678,13,4678,182,4678,72
2,5,22,981,510,X,26-35,Married,0.0,2.0,1.0,1.0,32.0,2350,145,2350,1,2350,587,11.40,97.493532,1459.71,229109.80,-178.10,-1.740068,0.0,-4089.16,-583.81,-29.953455,0.0,-70390.62,2350,8,2350,136,2350,108
3,8,25,1069,361,Y,18-25,Single,0.0,1.0,0.0,3.0,32.0,808,70,808,1,808,173,17.45,84.497970,497.97,68274.36,0.00,0.000000,0.0,0.00,-213.72,-16.059295,0.0,-12975.91,808,5,808,67,808,118
4,10,17,498,811,Y,46-55,Married,0.0,2.0,0.0,5.0,32.0,2201,129,2201,1,2201,396,10.69,106.768033,1022.65,234996.44,-35.62,-0.173953,0.0,-382.87,-400.37,-20.895557,0.0,-45991.12,2201,8,2201,121,2201,112


In [44]:
# Label encoding categorical columns
from sklearn import preprocessing

cat_columns = ['age_range', 'marital_status','campaign_type','rented','income_bracket']

for a_column_to_treat in cat_columns:
    le = preprocessing.LabelEncoder()
    le.fit(list(train_df[a_column_to_treat].astype(str).values) + list(test_df[a_column_to_treat].astype(str).values))
    train_df[a_column_to_treat] = le.transform(list(train_df[a_column_to_treat].astype(str).values))
    test_df[a_column_to_treat] = le.transform(list(test_df[a_column_to_treat].astype(str).values))

In [45]:
print(train_df.shape)
print(test_df.shape)

(78369, 37)
(50226, 36)


# Model start here

In [56]:
cat_params = {
        'learning_rate': 0.005,
        'bagging_temperature': 1, 
        'l2_leaf_reg': 3.0,
        'depth': 12, 
        'max_bin':255,
        'iterations' : 2000,
        #'task_type':'GPU',
        'loss_function' : "Logloss",
        'eval_metric' : "AUC",
        'bootstrap_type' : 'Bayesian',
        'random_seed':43,
        'early_stopping_rounds' : 100,
        'use_best_model': True 
}


In [57]:
   
import gc
SEED = 42
LOCAL_TEST=False
########################### Model
import catboost
from catboost import Pool
from sklearn.model_selection import StratifiedKFold

def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=2):
    N_SPLITS = NFOLDS
    #CV : using k-fold
    #folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
    
     #CV : using StratifiedKFold
    folds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  

    tt_df = tt_df[['id','campaign_id', 'coupon_id', 'customer_id', target]]    
    predictions = np.zeros(len(tt_df))
    tr_predictions = np.zeros(len(tr_df))
    
    categorical_features_indices = np.where(X.dtypes != np.float64)[0]
    print(X.columns)
    print(categorical_features_indices)    
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))        
        
        tr_data = Pool(tr_x,label=tr_y)
        vl_data = Pool(vl_x,label=vl_y)         

        estimator = catboost.CatBoostClassifier(**cat_params)        
        estimator.fit(tr_x, tr_y,  eval_set=(vl_x, vl_y), cat_features=categorical_features_indices, use_best_model=True, verbose = 300)

        #estimator = catboost.train(tr_data, cat_params, eval_set= vl_data, verbose = 300, init_model=catboost.CatBoostClassifier)
        
        pp_p = estimator.predict_proba(P)[:,1]
        # predictions += pp_p/NFOLDS
        predictions += pp_p/N_SPLITS
        #predictions = np.exp(predictions)/(1 + np.exp(predictions))
        
        # Include train prediction
        tr_p = estimator.predict_proba(X)[:,1]
        tr_predictions += tr_p/N_SPLITS
        #tr_predictions = np.exp(tr_predictions)/(1 + np.exp(tr_predictions))

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    tr_df['prediction'] = tr_predictions
    
    return tt_df, tr_df
## -------------------

In [58]:
feature_columns = list(train_df.columns)
feature_columns.remove('id')
feature_columns.remove('campaign_id')
feature_columns.remove('coupon_id')
feature_columns.remove('customer_id')
feature_columns.remove('redemption_status')
feature_columns.remove('prediction')
test_df['redemption_status']=0

In [60]:
test_predictions, train_predictions = make_predictions(train_df, test_df, feature_columns, 'redemption_status', cat_params, NFOLDS=4)

Index(['campaign_type', 'age_range', 'marital_status', 'rented', 'family_size',
       'no_of_children', 'income_bracket', 'campaign_duration',
       'J_group_name_count', 'J_group_name_nunique', 'J_customer_id_count',
       'J_customer_id_nunique', 'J_item_id_count', 'J_item_id_nunique',
       'J_selling_price_min', 'J_selling_price_mean', 'J_selling_price_max',
       'J_selling_price_sum', 'J_coupon_discount_min',
       'J_coupon_discount_mean', 'J_coupon_discount_max',
       'J_coupon_discount_sum', 'J_other_discount_min',
       'J_other_discount_mean', 'J_other_discount_max', 'J_other_discount_sum',
       'J_category_count', 'J_category_nunique', 'J_brand_type_count',
       'J_brand_type_nunique', 'J_date_count', 'J_date_nunique'],
      dtype='object')
[ 0  1  2  3  6  8  9 10 11 12 13 26 27 28 29 30 31]
Fold: 0
58776 19593
0:	test: 0.7779340	best: 0.7779340 (0)	total: 249ms	remaining: 8m 18s
300:	test: 0.8814781	best: 0.8814781 (300)	total: 1m 35s	remaining: 8m 58s
600:	

C:\Users\Nagesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [61]:
print(test_predictions.shape)
test_predictions.head()

(50226, 6)


,id,campaign_id,coupon_id,customer_id,redemption_status,prediction
0,3,22,869,967,0,0.087237
1,4,20,389,1566,0,0.077506
2,5,22,981,510,0,0.125309
3,8,25,1069,361,0,0.062087
4,10,17,498,811,0,0.066588


In [62]:
submission_df = (test_predictions.groupby(['id'])).max()
submission_df['id'] = submission_df.index
submission_df = submission_df[['id', 'prediction']]
submission_df = submission_df.rename(columns={'prediction': 'redemption_status'})

In [63]:
submission_df.head()

,id,redemption_status
id,,
3,3,0.087237
4,4,0.077506
5,5,0.125309
8,8,0.062087
10,10,0.066588


In [64]:
submission_df.to_csv("cat_submission4fold.csv", index=False)

In [46]:

lgb_params = { 'objective':'binary',
              'boosting_type':'gbdt',
              'metric':'auc',
              'n_jobs':-1,
              'learning_rate':0.005,
              'num_leaves': 2**8,
              'max_depth':-1,
              'tree_learner':'serial',
              'colsample_bytree': 0.7,
              'subsample_freq':1,
              'subsample':0.7,
              'n_estimators':3000,
              'max_bin':255,
              'verbose':-1,
              'seed': 42,
              'early_stopping_rounds':30
             } 


In [47]:
import gc
SEED = 42
LOCAL_TEST=False
########################### Model
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [48]:
def make_predictionsLgb(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=2):
    N_SPLITS = NFOLDS
    #CV : using k-fold
    #folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
    
     #CV : using StratifiedKFold
    folds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  

    tt_df = tt_df[['id','campaign_id', 'coupon_id', 'customer_id', target]]    
    predictions = np.zeros(len(tt_df))
    tr_predictions = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)

        if LOCAL_TEST:
            vl_data = lgb.Dataset(P, label=P_y) 
        else:
            vl_data = lgb.Dataset(vl_x, label=vl_y)  

        estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
        pp_p = estimator.predict(P)
        # predictions += pp_p/NFOLDS
        predictions += pp_p/N_SPLITS
        
        # Include train prediction
        tr_p = estimator.predict(X)
        tr_predictions += tr_p/N_SPLITS

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    tr_df['prediction'] = tr_predictions
    
    return tt_df, tr_df
## -------------------

In [49]:

feature_columns = list(train_df.columns)
feature_columns.remove('id')
feature_columns.remove('campaign_id')
feature_columns.remove('coupon_id')
feature_columns.remove('customer_id')
feature_columns.remove('redemption_status')
test_df['redemption_status']=0

In [50]:
test_predictions, train_predictions = make_predictionsLgb(train_df, test_df, feature_columns, 'redemption_status', lgb_params, NFOLDS=10)

Fold: 0
70532 7837


C:\Users\Nagesh\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Nagesh\Anaconda3\lib\site-packages\lightgbm\engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[37]	training's auc: 0.966001	valid_1's auc: 0.919726
Fold: 1
70532 7837
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[10]	training's auc: 0.964837	valid_1's auc: 0.872026
Fold: 2
70532 7837
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[44]	training's auc: 0.967505	valid_1's auc: 0.875979
Fold: 3
70532 7837
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[52]	training's auc: 0.967563	valid_1's auc: 0.906841
Fold: 4
70532 7837
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[26]	training's auc: 0.966154	valid_1's auc: 0.897105
Fold: 5
70532 7837
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[30]	training's auc: 0.966478	valid_1's auc: 0.870101

C:\Users\Nagesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [51]:
test_predictions.shape

(50226, 6)

In [52]:
test_predictions.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,prediction
0,3,22,869,967,0,0.012708
1,4,20,389,1566,0,0.008564
2,5,22,981,510,0,0.030983
3,8,25,1069,361,0,0.007762
4,10,17,498,811,0,0.007818


In [53]:
submission_df = (test_predictions.groupby(['id'])).max()
submission_df['id'] = submission_df.index
submission_df = submission_df[['id', 'prediction']]
submission_df = submission_df.rename(columns={'prediction': 'redemption_status'})


In [54]:
submission_df.head()

,id,redemption_status
id,,
3,3,0.012708
4,4,0.008564
5,5,0.030983
8,8,0.007762
10,10,0.007818


In [55]:
submission_df.to_csv("lgb_submission4fold.csv", index=False)